## This is to extract categories same day, 1-6 days, 7-30 days, or more than 30 days

In [1]:
import numpy as np
import pandas as pd
import joblib


In [2]:
data = pd.read_csv("algebra_2008_2009_train_trimmed3_remove_duplicates_sorted.csv", delimiter='\t')

In [3]:
data.describe(include='all')

,Anon Student Id,Problem Name,Problem View,Step Name,First Transaction Time,Correct First Attempt,Incorrects,Hints,Corrects,KC(SubSkills),Opportunity(SubSkills),KC(KTracedSkills),Opportunity(KTracedSkills),KC(Rules),Opportunity(Rules),Unit,Sections
count,8483920,8483920,8.483920e+06,8483920,8483920,8.483920e+06,8.483920e+06,8.483920e+06,8.483920e+06,6204248,6204248,4296385,4296385,8201432,8201432,8483920,8483920
unique,3310,188368,NaN,700635,823557,NaN,NaN,NaN,NaN,1815,910506,921,157173,2780,10780,42,165
top,stu_ea97537407,L5FB16,NaN,FinalAnswer,2009-04-27 12:55:20.0,NaN,NaN,NaN,NaN,Bogus skill,1,Entering a given-1,1,DISTANCE-MIDPOINT-PREFILLED-BOGUS-HINT,1,Unit CTA1_02,Section CTA1_02-2
freq,15608,52755,NaN,255762,109,NaN,NaN,NaN,NaN,408686,229924,440356,243040,408686,436844,715873,527203
mean,NaN,NaN,1.091596e+00,NaN,NaN,8.595012e-01,3.432128e-01,8.589826e-02,1.008225e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,3.611967e-01,NaN,NaN,3.475039e-01,3.904958e+00,4.622137e-01,3.963328e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,1.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data['First Transaction Time'] = pd.to_datetime(data['First Transaction Time'], format='%Y-%m-%d %H:%M:%S')

In [5]:
delta = data['First Transaction Time'][0] - data['First Transaction Time'][100]
delta.days

-6

In [6]:
# Build a dict of all ID-KC pairs -> to last date
from collections import deque


id_kc_to_hint = {}
for index, (stu_id, subskill, traced, rule) in enumerate(zip(data['Anon Student Id'],
                                                             data['KC(SubSkills)'], 
                                                             data['KC(KTracedSkills)'], 
                                                             data['KC(Rules)'])):
    
    if subskill is not np.nan:
        for kc in subskill.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc not in id_kc_to_hint:
                id_kc_to_hint[id_kc] = None
    
    if traced is not np.nan:
        for kc in traced.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc not in id_kc_to_hint:
                id_kc_to_hint[id_kc] = None
    
    if rule is not np.nan:
        for kc in rule.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc not in id_kc_to_hint:
                id_kc_to_hint[id_kc] = None
    
    if index%500000 == 0:
        print index

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000


In [7]:
# Build a dictionary mapping of id-unit pair -> [rows in testing data that hold the id-unit pair]
id_unit_dict = {}


for index, (stu_id, unit) in enumerate(zip(data['Anon Student Id'], data['Unit'])):
    id_unit = "".join([stu_id, unit])
    if id_unit not in id_unit_dict:
        id_unit_dict[id_unit] = []    
    if index%500000 == 0:
        print index

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000


In [8]:
# Get and process test data
testdata = pd.read_csv('algebra_2008_2009_test.txt', delimiter='\t')
split_object = testdata['Problem Hierarchy'].str.split(', ')
testdata['Unit'] = split_object.str[0]
testdata['Sections'] = split_object.str[1]
del testdata['Problem Hierarchy']

In [9]:
testdata.describe(include='all')

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Row,Anon Student Id,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),...,Hints,Corrects,KC(SubSkills),Opportunity(SubSkills),KC(KTracedSkills),Opportunity(KTracedSkills),KC(Rules),Opportunity(Rules),Unit,Sections
count,508912.000000,508912,508912,508912.000000,508912,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,353798,353798,229139,229139,493643,493643,508912,508912
unique,NaN,3243,11058,NaN,44982,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1270,113977,726,25926,1334,2972,42,165
top,NaN,stu_b83533cbf2,BH1T39,NaN,R3C2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Correctly placing points,7,Entering a given-1,3,UNIT-HELP,3,Unit CTA1_02,Section CTA1_06-3
freq,NaN,949,7641,NaN,13568,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,19334,5651,36065,6235,24317,12608,67166,27403
mean,254456.500000,NaN,NaN,1.117140,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,146910.384434,NaN,NaN,0.389737,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,127228.750000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,254456.500000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,381684.250000,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Fill in the values of id_unit_dict with the rows in test that contain the id-unit pair
for index, (stu_id, unit) in enumerate(zip(testdata['Anon Student Id'], testdata['Unit'])):
    id_unit = "".join([stu_id, unit])
    id_unit_dict[id_unit].append(index)

In [11]:
# These are the features
training_days_since_subskill = [None] * data['Anon Student Id'].size  # Days elapsed since student saw step w/ same KC components
testing_days_since_subskill = [None] * testdata['Anon Student Id'].size
training_days_since_traced = [None] * data['Anon Student Id'].size
testing_days_since_traced = [None] * testdata['Anon Student Id'].size
training_days_since_rules = [None] * data['Anon Student Id'].size
testing_days_since_rules = [None] * testdata['Anon Student Id'].size


In [12]:
# CREATE THE FEATURES FOR TRAINING AND TESTING IN ONE GO!!

previous_idunit = None
for index, (stu_id, unit, subskill, traced, rule, my_date) in enumerate(zip(data['Anon Student Id'], data['Unit'],
                                                                   data['KC(SubSkills)'], 
                                                                   data['KC(KTracedSkills)'], 
                                                                   data['KC(Rules)'], data['First Transaction Time'])):
    
    
    current_idunit = "".join([stu_id, unit])
    
    ## do updating of testdata here
    if previous_idunit is None:
        pass
    
    else:
        if previous_idunit != current_idunit:
            for idx in id_unit_dict[previous_idunit]:
                
                if testdata['KC(SubSkills)'][idx] is not np.nan:
                    for kc in testdata['KC(SubSkills)'][idx].split('~~'):
                        id_kc = "".join([testdata['Anon Student Id'][idx], kc])
                        try:
                            if id_kc_to_hint[id_kc]:
                                delta = previous_time - id_kc_to_hint[id_kc]
                                day_difference = delta.days
                                if testing_days_since_subskill[idx] is None or testing_days_since_subskill[idx] > day_difference:
                                    testing_days_since_subskill[idx] = day_difference
                        except KeyError:
                            pass
                
                if testdata['KC(KTracedSkills)'][idx] is not np.nan:
                    for kc in testdata['KC(KTracedSkills)'][idx].split('~~'):
                        id_kc = "".join([testdata['Anon Student Id'][idx], kc])
                        try:
                            if id_kc_to_hint[id_kc]:
                                delta = previous_time - id_kc_to_hint[id_kc]
                                day_difference = delta.days
                                if testing_days_since_traced[idx] is None or testing_days_since_traced[idx] > day_difference:
                                    testing_days_since_traced[idx] = day_difference
                        except KeyError:
                            pass
                        
                if testdata['KC(Rules)'][idx] is not np.nan:
                    for kc in testdata['KC(Rules)'][idx].split('~~'):
                        id_kc = "".join([testdata['Anon Student Id'][idx], kc])
                        try:
                            if id_kc_to_hint[id_kc]:
                                delta = previous_time - id_kc_to_hint[id_kc]
                                day_difference = delta.days
                                if testing_days_since_rules[idx] is None or testing_days_since_rules[idx] > day_difference:
                                    testing_days_since_rules[idx] = day_difference
                        except KeyError:
                            pass
                        
    previous_idunit = current_idunit
    previous_time = my_date
    
    
    if subskill is not np.nan:
        for kc in subskill.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc_to_hint[id_kc]:
                delta = my_date - id_kc_to_hint[id_kc]
                day_difference = delta.days
                if training_days_since_subskill[index] is None or training_days_since_subskill[index] > day_difference:
                    training_days_since_subskill[index] = day_difference
            id_kc_to_hint[id_kc] = my_date
    
    if traced is not np.nan:
        for kc in traced.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc_to_hint[id_kc]:
                delta = my_date - id_kc_to_hint[id_kc]
                day_difference = delta.days
                if training_days_since_traced[index] is None or training_days_since_traced[index] > day_difference:
                    training_days_since_traced[index] = day_difference
            id_kc_to_hint[id_kc] = my_date
    
    if rule is not np.nan:
        for kc in rule.split('~~'):
            id_kc = "".join([stu_id, kc])
            if id_kc_to_hint[id_kc]:
                delta = my_date - id_kc_to_hint[id_kc]
                day_difference = delta.days
                if training_days_since_rules[index] is None or training_days_since_rules[index] > day_difference:
                    training_days_since_rules[index] = day_difference
            id_kc_to_hint[id_kc] = my_date

    
    if index%500000 == 0:
        print index
    
    

for idx in id_unit_dict[previous_idunit]:
                
    if testdata['KC(SubSkills)'][idx] is not np.nan:
        for kc in testdata['KC(SubSkills)'][idx].split('~~'):
            id_kc = "".join([testdata['Anon Student Id'][idx], kc])
            try:
                if id_kc_to_hint[id_kc]:
                    delta = previous_time - id_kc_to_hint[id_kc]
                    day_difference = delta.days
                    if testing_days_since_subskill[idx] is None or testing_days_since_subskill[idx] > day_difference:
                        testing_days_since_subskill[idx] = day_difference
            except KeyError:
                pass

    if testdata['KC(KTracedSkills)'][idx] is not np.nan:
        for kc in testdata['KC(KTracedSkills)'][idx].split('~~'):
            id_kc = "".join([testdata['Anon Student Id'][idx], kc])
            try:
                if id_kc_to_hint[id_kc]:
                    delta = previous_time - id_kc_to_hint[id_kc]
                    day_difference = delta.days
                    if testing_days_since_traced[idx] is None or testing_days_since_traced[idx] > day_difference:
                        testing_days_since_traced[idx] = day_difference
            except KeyError:
                pass

    if testdata['KC(Rules)'][idx] is not np.nan:
        for kc in testdata['KC(Rules)'][idx].split('~~'):
            id_kc = "".join([testdata['Anon Student Id'][idx], kc])
            try:
                if id_kc_to_hint[id_kc]:
                    delta = previous_time - id_kc_to_hint[id_kc]
                    day_difference = delta.days
                    if testing_days_since_rules[idx] is None or testing_days_since_rules[idx] > day_difference:
                        testing_days_since_rules[idx] = day_difference
            except KeyError:
                pass



0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000


In [ ]:
del id_kc_to_hint

## Convert to file

In [35]:
from scipy.sparse import coo_matrix, hstack

In [38]:
row = []
col = []
values = []

for index, day in enumerate(training_days_since_subskill):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

training_day_since_subskill_sparse = coo_matrix((values, (row, col)), shape=(len(training_days_since_subskill), 4))
training_day_since_subskill_sparse

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


<8483920x4 sparse matrix of type '<type 'numpy.float64'>'
	with 5945867 stored elements in COOrdinate format>

In [40]:
row = []
col = []
values = []

for index, day in enumerate(training_days_since_traced):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

training_day_since_traced_sparse = coo_matrix((values, (row, col)), shape=(len(training_days_since_traced), 4))
training_day_since_traced_sparse

<8483920x4 sparse matrix of type '<type 'numpy.float64'>'
	with 4044138 stored elements in COOrdinate format>

In [41]:
row = []
col = []
values = []

for index, day in enumerate(training_days_since_rules):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

training_day_since_rules_sparse = coo_matrix((values, (row, col)), shape=(len(training_days_since_rules), 4))
training_day_since_rules_sparse

<8483920x4 sparse matrix of type '<type 'numpy.float64'>'
	with 7740456 stored elements in COOrdinate format>

In [42]:
train_recency_features = hstack([training_day_since_subskill_sparse, 
                                 training_day_since_traced_sparse, training_day_since_rules_sparse])
train_recency_features

<8483920x12 sparse matrix of type '<type 'numpy.float64'>'
	with 17730461 stored elements in COOrdinate format>

In [43]:
row = []
col = []
values = []

for index, day in enumerate(testing_days_since_subskill):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

testing_day_since_subskill_sparse = coo_matrix((values, (row, col)), shape=(len(testing_days_since_subskill), 4))
testing_day_since_subskill_sparse

<508912x4 sparse matrix of type '<type 'numpy.float64'>'
	with 348497 stored elements in COOrdinate format>

In [44]:
row = []
col = []
values = []

for index, day in enumerate(testing_days_since_traced):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

testing_day_since_traced_sparse = coo_matrix((values, (row, col)), shape=(len(testing_days_since_traced), 4))
testing_day_since_traced_sparse

<508912x4 sparse matrix of type '<type 'numpy.float64'>'
	with 222881 stored elements in COOrdinate format>

In [45]:
row = []
col = []
values = []

for index, day in enumerate(testing_days_since_rules):
    if day is None:
        pass
    elif day == 0:
        col.append(0)
        values.append(1.)
        row.append(index)
    elif day <= 7:
        col.append(1)
        values.append(1.)
        row.append(index)
    elif day <= 30:
        col.append(2)
        values.append(1.)
        row.append(index)
    else:
        col.append(3)
        values.append(1.)
        row.append(index)

testing_day_since_rules_sparse = coo_matrix((values, (row, col)), shape=(len(testing_days_since_rules), 4))
testing_day_since_rules_sparse

<508912x4 sparse matrix of type '<type 'numpy.float64'>'
	with 474463 stored elements in COOrdinate format>

In [46]:
test_recency_features = hstack([testing_day_since_subskill_sparse, 
                                 testing_day_since_traced_sparse, testing_day_since_rules_sparse])
test_recency_features

<508912x12 sparse matrix of type '<type 'numpy.float64'>'
	with 1045841 stored elements in COOrdinate format>

In [47]:
import joblib

In [48]:
joblib.dump(train_recency_features, 'algebra_2005_2008_pickles/train_recency_features', compress=3)

['algebra_2005_2008_pickles/train_recency_features']

In [49]:
joblib.dump(test_recency_features, 'algebra_2005_2008_pickles/test_recency_features', compress=3)

['algebra_2005_2008_pickles/test_recency_features']